# **Schema Generation**

Here we will generate a schema and try to place some reasonable constraints on the data. 

We will create the schema based on all of the available data. For now, the schema will only have information about the tags. 

In [22]:
import os
import pprint
import tempfile
import urllib
import numpy as np

import pandas as pd

import absl
import tensorflow as tf
import tensorflow_data_validation as tfdv
tf.get_logger().propagate = False
pp = pprint.PrettyPrinter()

import tfx
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import ExampleValidator
from tfx.components.base import executor_spec
from tfx.components.trainer.executor import GenericExecutor
from tfx.dsl.experimental import latest_blessed_model_resolver
from tfx.orchestration import metadata
from tfx.orchestration import pipeline
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.extensions.google_cloud_big_query.example_gen.component import (
    BigQueryExampleGen,
)

from tfx.components import ImporterNode
from tfx.types.standard_artifacts import Schema

from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2
from tfx.proto import example_gen_pb2

from tensorflow_metadata.proto.v0 import schema_pb2 
from tensorflow_metadata.proto.v0 import statistics_pb2
from tensorflow_metadata.proto.v0 import anomalies_pb2

from ml_metadata import metadata_store
from ml_metadata.proto import metadata_store_pb2

from tfx.types import Channel
from tfx.types.standard_artifacts import Model
from tfx.types.standard_artifacts import ModelBlessing
from tfx.utils.dsl_utils import external_input

In [2]:
_pipeline_root = tempfile.mkdtemp(prefix='pipeline')
_pipeline_name = 'interactive_pipeline'

context = InteractiveContext(pipeline_name=_pipeline_name, pipeline_root=_pipeline_root)

We are only querying 100 rows in order to get the types for the different columns we will use during training. We will generate the vocabulary for the tags separately (outside of tfx), because in order to generate the vocabulary for the tags within tfx we would need to generate tfrecords for every single example. This is unnecessary, we can generate a schema from a small amount of data first (mostly to get the types for all features) and then separately generate the vocab and set the domain of the tags manually. 

See below for how this is accomplished. 

In [3]:
query = """
SELECT 
    synopsis,
    tags
FROM 
    `metadata_sky.merlin_movie_series_data_small`
LIMIT 100
"""

In [4]:
%%time
output = example_gen_pb2.Output(
             split_config=example_gen_pb2.SplitConfig(splits=[
                 example_gen_pb2.SplitConfig.Split(name='train', hash_buckets=10)
             ],
    ))
example_gen = BigQueryExampleGen(query=query, output_config=output)
context.run(example_gen, beam_pipeline_args=['--project', 'ml-sandbox-101', '--temp_location', 'gs://metadata-bucket-sky/tmp'])

/opt/conda/lib/python3.7/site-packages/apache_beam/io/gcp/bigquery.py:1881: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  temp_location = pcoll.pipeline.options.view_as(


CPU times: user 836 ms, sys: 146 ms, total: 982 ms
Wall time: 17.8 s


ExecutionResult(
    component_id: BigQueryExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 1
        type_id: 5
        uri: "/tmp/pipelinek433r6ld/BigQueryExampleGen/examples/1"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "examples"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "BigQueryExampleGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 5
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
        ))

**Inspect the data**

In [5]:
def make_dataset(examples, split):
    """
    Create a dataset from examples artifact
    """
    uri = os.path.join(examples.get()[0].uri, split)
    
    tfrecord_filenames = [os.path.join(uri, name) for name in os.listdir(uri)]
    dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")
    
    return dataset

In [6]:
train_dataset = make_dataset(example_gen.outputs['examples'], 'train')

print('TRAIN DATASET SIZE: {}'.format(train_dataset.reduce(0, lambda x, _: x + 1)))

print('{:=^80}'.format(' Example '))

# Iterate over the first record and decode it.
for tfrecord in train_dataset.take(1):
    serialized_example = tfrecord.numpy()
    example = tf.train.Example()
    example.ParseFromString(serialized_example)
    pp.pprint(example)

TRAIN DATASET SIZE: 100
=================================== Example ====================================
features {
  feature {
    key: "synopsis"
    value {
      bytes_list {
        value: "Soccer players Pablo Mastroeni and Jimmy Conrad join Emeril in the kitchen. Lamb and white bean casserole; cowboy chicken casserole. Homemade bagels; kicked-up schmeers; New Orleans crawfish bagels; bagel chips. Dad\'s day off. Barbecue favorites. Artichokes stuffed with ricotta; salami rolls; fried turkey; chocolate-mousse layer cake. Bloody Maria; warm hearts of palm salad; broiled crab backs; fettuccini with shellfish. Salad; cold puree of white bean soup; fried green tomato and rock-shrimp salad; roasted"
      }
    }
  }
  feature {
    key: "tags"
    value {
      bytes_list {
        value: "Food"
        value: "How-To"
      }
    }
  }
}



**The following query allows us to get the entire tag vocab**. This takes about 1 second to run, as opposed to several minutes to generate examples for every row data and then calculate the statistics for that data. 

In [7]:
%%bigquery tag_vocab_df

SELECT 
  DISTINCT(labels)
FROM `ml-sandbox-101.metadata_sky.merlin_movie_series_data_small`, UNNEST(tags) as labels

Unfortunately, this also causes us to calculate a domain for things like synopsis and content_id, which we don't want. 

In [9]:
stats_options = tfdv.StatsOptions(num_rank_histogram_buckets=1000)
statistics_gen = StatisticsGen(
    examples=example_gen.outputs['examples'],
    stats_options=stats_options)
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(artifact: id: 2
        type_id: 7
        uri: "/tmp/pipelinek433r6ld/StatisticsGen/statistics/2"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "statistics"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "StatisticsGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 7
        name: "ExampleStatistics"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
        ))

Let's look at the statistics:

In [10]:
context.show(statistics_gen.outputs['statistics'])

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Some useful stuff here: 
 + Data is never missing (constraint)
 + Can potentially do some thresholds on labels + tags in order to detect drift 
     + Hard to say what is reasonable here, but we can just start with some value and see what happens
 + Can set the label and tag vocabulary if we'd like ("domain") 
     + Can also set constraints on the domain --> e.g. labels should always come from the observed domain. 
 + Need to increase the number of rank_histogram_buckets. 

In [11]:
stats_path = statistics_gen.outputs['statistics'].get()[0].uri
train_statistics_path = os.path.join(stats_path, 'train/stats_tfrecord')

train_statistics = tfdv.load_statistics(train_statistics_path)

In [16]:
# Get the full tag domain by setting max_string_domain_size to 5000
schema = tfdv.infer_schema(train_statistics, infer_feature_shape=True, max_string_domain_size=50)

In [17]:
tfdv.display_schema(schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'synopsis',BYTES,required,,-
'tags',BYTES,required,"[1,inf)",-


**Below is where we set the domain manually**

In [24]:
tags = tfdv.get_feature(schema, 'tags')
tfdv.set_domain(schema, 'tags', schema_pb2.StringDomain(value=np.hstack(tag_vocab_df.values)))

1
4


In [29]:
tags.distribution_constraints.min_domain_mass = 0.95
tags.drift_comparator.infinity_norm.threshold = 0.05

In [30]:
tfdv.write_schema_text(schema, '../src/schema/schema.pbtxt')

---

In [31]:
loaded_schema = tfdv.load_schema_text('../src/schema/schema.pbtxt')

In [32]:
def get_domain_size(schema_path, feature):
    schema_text = tfdv.load_schema_text(schema_path)
    domain = tfdv.get_domain(schema_text, feature)

    return len(domain.value)

Can confirm that the schema has the entire domain as well as both features and their type! 

In [34]:
tfdv.display_schema(loaded_schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'synopsis',BYTES,required,,-
'tags',STRING,required,"[1,inf)",'tags_domain'


/opt/conda/lib/python3.7/site-packages/tensorflow_data_validation/utils/display_util.py:151: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


Values
Domain                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [33]:
get_domain_size('../src/schema/schema.pbtxt', 'tags')

409